<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER/blob/main/Imocap_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
pip install transformers datasets evaluate

In [16]:
from huggingface_hub import notebook_login

notebook_login()

# **Loading Base wav2vec2 model**

In [17]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:306: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [18]:
%%capture
!pip install gdown

In [19]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [20]:
%%capture
!pip install kaggle

In [21]:

%%capture
!pip install -U transformers[torch] datasets


%%capture
!pip install tensorboards

In [ ]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

print("Upload the kaggle.json file")
# Ensure files.upload() is called in the same cell as its import
files.upload() # Calling files.upload() immediately after import
time.sleep(5)

os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
time.sleep(5)

!kaggle datasets download -d dejolilandry/iemocapfullrelease
time.sleep(5)

with zipfile.ZipFile('iemocapfullrelease.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')
time.sleep(5)

!gdown --id 16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
time.sleep(5)

!unzip /content/IEMOCAP_splits.zip
time.sleep(5)

Upload the kaggle.json file


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/dejolilandry/iemocapfullrelease
License(s): unknown
iemocapfullrelease.zip: Skipping, found more recently modified local copy (use --force to force download)
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
To: /content/IEMOCAP_splits.zip
100% 817k/817k [00:00<00:00, 108MB/s]
Archive:  /content/IEMOCAP_splits.zip
   creating: iemocap/
   creating: iemocap/session5/
  inflating: iemocap/session5/test.json  
  inflating: iemocap/session5/train.json  
   creating: iemocap/session2/
  inflating: iemocap/session2/test.json  
  inflating: iemocap/session2/train.js

In [9]:
import shutil
import os

# Define the current and target paths
current_path = "/content/iemocap"  # Current location of IEMOCAP folder
target_path = "/content/IEMOCAP_full_release/iemocap"   # New target location

# Create target directory if it doesn't exist
os.makedirs(os.path.dirname(target_path), exist_ok=True)

# Move the folder
shutil.move(current_path, target_path)
print("IEMOCAP folder successfully moved to:", target_path)


IEMOCAP folder successfully moved to: /content/IEMOCAP_full_release/iemocap


In [27]:
import json
import os

# Define the prefix to remove
prefix_to_remove = "/m/teamwork/t40511_asr/c/"

# Function to update file paths in the JSON data
def update_file_paths(json_file_path, updated_json_file_path):
    # Open and load the existing JSON file
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    # Loop through all sessions and their data in the JSON
    for session, session_data in data.items():
        # Check if 'wav' starts with the specified prefix and remove it
        if session_data["wav"].startswith(prefix_to_remove):
            session_data["wav"] = session_data["wav"][len(prefix_to_remove):]  # Remove the prefix

    # Save the updated JSON data into a new file
    with open(updated_json_file_path, 'w') as file:
        json.dump(data, file, indent=4)

# Define the path to the iemocap directory
iemocap_dir = "/content/IEMOCAP_full_release/iemocap"

# Loop through all session directories (session1 to session5)
for session_num in range(1, 6):
    session_dir = os.path.join(iemocap_dir, f"session{session_num}")

    # Define the paths for the train and test JSON files
    train_json_path = os.path.join(session_dir, "train.json")
    test_json_path = os.path.join(session_dir, "test.json")

    # Define the updated JSON file paths
    updated_train_json_path = os.path.join(session_dir, "updated_train.json")
    updated_test_json_path = os.path.join(session_dir, "updated_test.json")

    # Update the file paths in both train and test JSON files
    update_file_paths(train_json_path, updated_train_json_path)
    update_file_paths(test_json_path, updated_test_json_path)

print("File paths updated successfully for all sessions!")


File paths updated successfully for all sessions!


**FOR SESSION1**


In [28]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/IEMOCAP_full_release/iemocap/session1/updated_train.json",'r') as f:
    train_data = json.load(f)

with open("/content/IEMOCAP_full_release/iemocap/session1/updated_test.json",'r') as f:
    test_data = json.load(f)



In [29]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [30]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)

In [31]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 4446
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1085
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1085
    })
})

In [32]:

%%capture
!git clone https://github.com/aalto-speech/mtkd4ser.git

In [33]:
import os

def add_base_path(example):
    # Assuming the original paths are relative to /content/iemocap/
    example['wav'] = os.path.join('/content/', example['wav'])
    return example

dataset = dataset.map(add_base_path, batched=False)
dataset['train'][0]

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

{'id': 'Ses02F_impro03_F000',
 'wav': '/content/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav',
 'length': 4.0475736961451245,
 'emo': 'hap'}

In [34]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [35]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 4446
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 1085
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 1085
    })
})

In [36]:
dataset['train'][0].keys()

dict_keys(['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'])

In [37]:
!pip install tensorboard
from torch.utils.tensorboard import SummaryWriter
!pip install tqdm
from tqdm import tqdm

In [2]:
# Metrics functions
def calculate_unweighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='macro')

def calculate_weighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='weighted')

def calculate_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

# Collate function for DataLoader
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: len(x["input_values"]), reverse=True)
    inputs = [torch.tensor(example["input_values"]) for example in batch]
    labels = [int(example["label"]) for example in batch]
    inputs = pad_sequence(inputs, batch_first=True)
    return {"input_values": inputs, "label": torch.tensor(labels)}

# Example helper function to update paths
def update_iemocap_path(path):
    return "/content/" + path  # Adjust as necessary for your file path

In [3]:
!pip install transformers datasets
import torch
from torch.nn.utils.rnn import pad_sequence

In [4]:
def update_iemocap_label(label, column="emo"):
    if label == "ang":
        return "anger"
    elif label == "hap":
        return "happiness"
    elif label == "neu":
        return "neutral"
    elif label == "sad":
        return "sadness"

In [5]:
def to_label(emo, label2id):
    return label2id[emo]

In [6]:
def is_common(emotion):
    if emotion == "neutral" or emotion == "sadness" or emotion == "happiness" or emotion == "anger":
        return 1
    else:
        return 0

In [7]:
def iemocap(session):
    train_json_file = f"/content/IEMOCAP_full_release/iemocap/session{session}/updated_train.json"
    test_json_file = f"/content/IEMOCAP_full_release/iemocap//session{session}/updated_test.json"

    with open(train_json_file, "r") as f:
        train_data = json.load(f)
    with open(test_json_file, "r") as f:
        test_data = json.load(f)

    train_df = pd.DataFrame.from_dict(train_data, orient="index").reset_index()
    train_df = train_df.rename(columns={'index': 'id', 'wav': 'audio'})

    test_df = pd.DataFrame.from_dict(test_data, orient="index").reset_index()
    test_df = test_df.rename(columns={'index': 'id', 'wav': 'audio'})

    train_df["audio"] = train_df["audio"].apply(update_iemocap_path).values
    test_df["audio"] = test_df["audio"].apply(update_iemocap_path).values

    train_df["emo"] = train_df["emo"].apply(update_iemocap_label)
    test_df["emo"] = test_df["emo"].apply(update_iemocap_label)

    labels = ['anger', 'happiness', 'neutral', 'sadness'] #["ang", "hap", "neu", "sad"]
    label2id, id2label = dict(), dict()

    for i, label in enumerate(labels):
        label2id[label] = str(i)
        id2label[str(i)] = label

    # convert categorical label to numerical labels
    train_df["label"] = train_df["emo"].apply(to_label, args=(label2id,))
    test_df["label"] = test_df["emo"].apply(to_label, args=(label2id,))

    train_audio_data = Dataset.from_pandas(train_df[['audio', 'label']])
    train_audio_data = train_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    test_audio_data = Dataset.from_pandas(test_df[['audio', 'label']])
    test_audio_data = test_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    ds = DatasetDict({
        'train' : train_audio_data,
        'test' : test_audio_data,
        'dev' : test_audio_data,
    })

    return label2id, id2label, ds

In [8]:
!pip install accelerate
import random # import the random module
import numpy as np
import torch
from datasets import load_dataset

from transformers import Wav2Vec2FeatureExtractor
from torch.utils.data import DataLoader
import os
from sklearn.metrics import recall_score, accuracy_score
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [10]:
def preprocess_function(examples, feature_extractor):
    max_duration = 20.0
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=int(feature_extractor.sampling_rate * max_duration), padding=True, truncation=True)
    return inputs


In [11]:
def model_ft(label2id, id2label, num_classes, device="cpu"):
    model_name = "facebook/wav2vec2-base"
    model = AutoModelForAudioClassification.from_pretrained(model_name, num_labels=num_classes, label2id=label2id, id2label=id2label)
    model.to(device)
    return model

def pipeline_ft(model, train_loader, valid_loader, test_loader, hyperparam, device):
    SESSION = hyperparam['SESSION']
    LEARNING_RATE = hyperparam['LEARNING_RATE']
    TRAINING = hyperparam['TRAINING']
    N_EPOCHS = hyperparam['N_EPOCHS']
    LINGUALITY = hyperparam['LINGUALITY']
    LANGUAGE = hyperparam['LANGUAGE']

    file_path = f"/content/FT_{LINGUALITY}_{LANGUAGE}_S{SESSION}.pth"
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    # Load checkpoint if exists
    epoch = 1
    if os.path.exists(file_path):
        checkpoint = torch.load(file_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        total_train_loss = checkpoint.get('training loss', 0)
        total_valid_loss = checkpoint.get('validation loss', 0)
        print("Model checkpoint has been loaded")

    # Initialize TensorBoard writer
    writer = SummaryWriter(log_dir=f"/content/tensorboards/FT_{LINGUALITY}_{LANGUAGE}_S{SESSION}")

    # Evaluation-only mode
    if TRAINING == 0:
        test_unweighted_recall, test_weighted_recall, test_accuracy = validation(
            model,
            tqdm(test_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Test", leave=False),
            loss_fn,
            device
        )
        print(f"Epoch {epoch}/{N_EPOCHS}, Test Recall (unweighted): {test_unweighted_recall}, Recall (weighted): {test_weighted_recall}, Accuracy: {test_accuracy}\n")

    else:
        # Training and validation mode
        for epoch in range(epoch, N_EPOCHS+1):
            # Train step - adjust to returned values
            train_unweighted_recall, train_weighted_recall, train_accuracy = train(
                model,
                tqdm(train_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Training", leave=False),
                optimizer,
                loss_fn,
                device
            )
            print(f"Epoch {epoch}/{N_EPOCHS}, Training Recall (unweighted): {train_unweighted_recall}, Recall (weighted): {train_weighted_recall}, Accuracy: {train_accuracy}")

            # Log training metrics
            writer.add_scalar("Accuracy/Train", train_accuracy, epoch)
            writer.add_scalar("Recall_Unweighted/Train", train_unweighted_recall, epoch)
            writer.add_scalar("Recall_Weighted/Train", train_weighted_recall, epoch)

            # Validation step - adjust to returned values
            valid_unweighted_recall, valid_weighted_recall, valid_accuracy = validation(
                model,
                tqdm(valid_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Validation", leave=False),
                loss_fn,
                device
            )
            print(f"Epoch {epoch}/{N_EPOCHS}, Validation Recall (unweighted): {valid_unweighted_recall}, Recall (weighted): {valid_weighted_recall}, Accuracy: {valid_accuracy}\n")

            # Log validation metrics
            writer.add_scalar("Accuracy/Validation", valid_accuracy, epoch)
            writer.add_scalar("Recall_Unweighted/Validation", valid_unweighted_recall, epoch)
            writer.add_scalar("Recall_Weighted/Validation", valid_weighted_recall, epoch)

            writer.flush()

            # Save model checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'training loss': train_unweighted_recall,  # Adjust as needed
                'validation loss': valid_unweighted_recall  # Adjust as needed
            }, file_path)

            print(f"Model has been saved after epoch: {epoch}\n")

        # Close TensorBoard writer
        writer.close()

def train(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_train_loss, correct_predictions, total_predictions = 0, 0, 0
    all_actual_labels, all_predictions = [], []

    for batch in train_loader:
        inputs, labels = batch['input_values'].to(device), batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs).logits
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)
        all_actual_labels.extend(labels.tolist())
        all_predictions.extend(predicted.tolist())

    unweighted_recall = recall_score(all_actual_labels, all_predictions, average='macro')
    return unweighted_recall, correct_predictions / total_predictions, total_train_loss / len(train_loader)
# Define the validation function
def validation(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in data_loader:
            inputs, labels = batch['input_values'].to(device), batch['label'].to(device)
            outputs = model(inputs).logits
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            predictions = torch.argmax(outputs, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    unweighted_recall = recall_score(all_labels, all_predictions, average='macro')
    weighted_recall = recall_score(all_labels, all_predictions, average='weighted')
    accuracy = sum(np.array(all_labels) == np.array(all_predictions)) / len(all_labels)

    return unweighted_recall, weighted_recall, accuracy

In [12]:
# Define the Args class with instance variables
class Args:
    def __init__(self):
        self.N_EPOCHS = 10
        self.LEARNING_RATE = 1e-5
        self.BATCH_SIZE = 8
        self.SESSION = 1
        self.TRAINING = 1
        self.PARADIGM = "FT"
        self.LANGUAGE = "FR"
        self.LINGUALITY = "Monolingual"


In [13]:
def main(args_dict):
    random_seed = 42
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(random_seed)

    # Access values using args_dict
    session = args_dict["SESSION"]
    batch_size = args_dict["BATCH_SIZE"]
    learning_rate = args_dict["LEARNING_RATE"]

    # Load the CaFE dataset
    label2id, id2label, ds = iemocap(session)

    # Initialize feature extractor
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")

    # Preprocess dataset
    encoded_audio = ds.map(lambda example: preprocess_function(example, feature_extractor), remove_columns="audio", batched=True)

    # Initialize DataLoaders
    train_loader = DataLoader(encoded_audio["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(encoded_audio["test"], batch_size=batch_size, collate_fn=collate_fn)
    valid_loader = DataLoader(encoded_audio["dev"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # Model and optimizer
    model = model_ft(label2id, id2label, len(label2id), device="cuda" if torch.cuda.is_available() else "cpu")
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    # Run pipeline
    pipeline_ft(model, train_loader, valid_loader, test_loader, args_dict, device="cuda" if torch.cuda.is_available() else "cpu")



In [14]:
# Create an instance of Args and pass it as a dictionary to main
args = Args()
main(args.__dict__)

NameError: name 'json' is not defined